# Seq2Seq para tradução inglês - Português

Vamos implementar um modelo de seq2seq simples para traduzir texto de inglês para português carácter a carácter

In [1]:
from keras.models import Model
from keras.layers import Input, CuDNNLSTM, Dense, LSTM
import numpy as np

latent_dim = 512  #Tamanho do estado h
num_samples = 100000  # Numero de amostras a serem usadas


#Arquivo com pares inglês-português
data_path = 'por.txt'


Using TensorFlow backend.
/Users/lucasosouza/anaconda/envs/udacity/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
lines = open(data_path, encoding="utf8").read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # Usaremos um \t para indicar o inicio de uma frase e \n para indicar o fim
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Número de amostras:', len(input_texts))
print('Número de tokens unicos de input:', num_encoder_tokens)
print('Number de tokens unicos de output:', num_decoder_tokens)
print('Tamanho máximo de input:', max_encoder_seq_length)
print('Tamanho máximo de output:', max_decoder_seq_length)

#input_texts = input_texts[:5000]
#target_texts = target_texts[:5000]

#cria dicionário de tokens
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

#cria arrays de 0 para fazer o padding
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

#preenche os arrays com os caracteres
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data é a saída esperada do decoder, então deve estar um tempo a frente e não incluir o \t
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

Número de amostras: 100000
Número de tokens unicos de input: 86
Number de tokens unicos de output: 111
Tamanho máximo de input: 44
Tamanho máximo de output: 80


## Seq2Seq

Aqui criamos um modelo Seq2Seq simples com uma camada de LSTM como encoder e outra como decoder.

O encoder retorna seus estados que serão a inicialização do decoder

In [3]:
from keras.models import Sequential, Model


encoder_inputs = Input(shape=(None, num_encoder_tokens))

#Encoder
encoder = LSTM(latent_dim, return_state=True)

#pega saida do encoder
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]


decoder_inputs = Input(shape=(None, num_decoder_tokens))

#Decoder
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)

#pega saída do decoder
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)

#camada Dense com softmax do tamanho do vocabulario para definir qual o carácter de saída
decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

#O modelo para o treino tem como entrada o input a ser codificado e o input do Force Teaching do decoder
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [5]:
model.summary()

#Pesos do treino já executado
# model.load_weights('s2s_Samsung_carac.h5')
model.load_weights('seq2seq_weights.h5')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 86)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 111)    0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 512), (None, 1226752     input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 512),  1277952     input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

In [ ]:
from keras.callbacks import ModelCheckpoint
batch_size = 32  # Batch pequeno
epochs = 100


# Executa o treino
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs, shuffle=True, #Sem shuffle para não misturar frases de tamanhos diferentes
          validation_split=0.01, verbose = 1)

Train on 99000 samples, validate on 1000 samples
Epoch 1/100
 5504/99000 [>.............................] - ETA: 29:42 - loss: 0.1854

In [ ]:
model.save_weights('seq2seq_weights.h5')

Aqui vamos criar o modelo para executar testes

A principal diferença é que no treino temos o force teaching e no teste iremos usar a real saída do decoder como input

In [ ]:
#Modelo do Encoder
encoder_model = Model(encoder_inputs, encoder_states)


#Inputs para o decoder
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

#Reutiliza a LSTM treinada anteriormente com entradas novas
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

#Agora nosso modelo possui como entrada os estados que irão ser gerados pelo encoder
#e o output do decoder gerado pelos tempos anteriores

#Como saída temos também os estados para garantir que no próximo tempo o Decoder continue de onde parou
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)


#Cria dicionário de tokens inverso para decodificar a saída
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


A função abaixo irá fazer a geração de saída do decoder, pegando uma saída por vez e realimentando o Decoder

In [ ]:
def decode_sequence(input_seq):
    # Aplica as entradas no encoder para pegar seus estados (sua memória)
    states_value = encoder_model.predict(input_seq)

    # Gera um vetor de entrada falso para o decoder
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # coloca o \t na primeira posição do vetor indicando para o decoder que deve iniciar a gerar texto
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Faz o loop gerando saídas do Decoder
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        
        # Pega o token de saída
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        # Traduz para texto
        sampled_char = reverse_target_char_index[sampled_token_index]
        # Adiciona à string de resposta
        decoded_sentence += sampled_char

        # Se a saída for \n ou tiver atingido o tamanho máximo, para o loop
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Recria o input do decoder com a última saída
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence


for seq_index in range(5):
    # Aqui testamos as primeiras 5 entradas do dataset
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input:', input_texts[seq_index])
    print('Output:', decoded_sentence)

In [ ]:
def teste(teste):
    texto_seq = np.zeros((1,max_encoder_seq_length,num_encoder_tokens))

    for t, char in enumerate(teste):
        texto_seq[0, t, input_token_index[char]] = 1.
    
    decoded_sentence = decode_sequence(texto_seq)
    print('-')
    print('Input:', teste)
    print('Output:', decoded_sentence)

In [ ]:
teste('You\'ll do it now!')

In [ ]:
teste('Are you wrong')

In [ ]:
teste('You are wrong')

In [ ]:
teste('You don\'t look happy')

In [ ]:
teste('I hate you all')

In [ ]:
teste('Can you help me?')

In [ ]:
teste('Cant you bring it?')